In [5]:
import numpy as np

In [6]:
'''This function places the graph into a breadth-first structure'''
def breadth_first(graph, root):
    n = graph.shape[0]
    
    new_graph = -np.ones((n, n))
    level = -np.ones(n,int)
    nodes = np.array(range(n))#номера вершин для удобства
    
    current_level = 0
    
    level[root] = current_level
    
    while current_level in level:
        for node in nodes[level == current_level]:#не в тот массив смотрели(нужны были номера вершин!)
        
            
            index = []
            
            for i in range(n):
                if graph[node][i]>=0 and level[i] == -1:
                    index.append(i)
                    
            new_graph[node][index] = 0
            for i in index:#добавил симметрию
                new_graph[i][node] = 0
                
            level[index] = current_level + 1
            
        current_level += 1
    return new_graph, level

In [ ]:
def categorize(graph):
    n = graph.shape[0]
    
    color = -np.ones(n, int) #fill COLOR with -1
    current_color = 0
    nodes = np.array(range(n)) 
    
    while -1 in color:
        first = list(color).index(-1) #Finds the first occurence of -1 in the list
        index = [first]
        
        while len(index) != 0:
            index2 = []
            
            for node in index:

                for i in range(n):
                    if graph[node][i]>=0 and color[i] == -1:
                        index2.append(i)
                        color[i] = current_color
            
            index = index2
        current_color += 1
        
    return color
            

In [105]:
graph = np.array([[0, 1., 1., 0, 0],
       [1., 0, 1., 0, 0],
       [1., 1., 0, 1., 0],
       [0, 0, 1., 0, 1.],
       [0, 0, 0, 1., 0]])

In [106]:
graph[graph == 0] = -1

In [107]:
graph[graph>0] = 0

In [108]:
graph

array([[-1.,  0.,  0., -1., -1.],
       [ 0., -1.,  0., -1., -1.],
       [ 0.,  0., -1.,  0., -1.],
       [-1., -1.,  0., -1.,  0.],
       [-1., -1., -1.,  0., -1.]])

In [11]:
root = 0

In [12]:
new_graph, level = breadth_first(graph,root)

In [13]:
new_graph

array([[-1.,  0.,  0., -1., -1.],
       [ 0., -1., -1., -1., -1.],
       [ 0., -1., -1.,  0., -1.],
       [-1., -1.,  0., -1.,  0.],
       [-1., -1., -1.,  0., -1.]])

In [14]:
list(range(int(level.max()+1)))

[0, 1, 2, 3]

breadth_first - вроде исправил

In [51]:
'''This function calculates the values of the shortest paths to each node (shortest path = the sum of incoming paths)'''
def pointer(new_graph, level, root):
    
    n = new_graph.shape[0]
    output = np.zeros(n)
    output[root] = 1
    nodes = np.array(range(n))#номера вершин для удобства
    
    for x in range(int(level.max()+1)):
        for y in nodes[level == x]: #upside down #не в тот массив смотрели(нужны были номера вершин!)
            index = []
            for i in range(n): 
                if new_graph[y][i] >= 0 and level[i] == x+1:
                    index.append(i)

            output[index] += output[y]
    
    
    return output

In [16]:
output = pointer(new_graph, level, root)

In [17]:
np.array(range(np.max(level)+1)[::-1])

array([3, 2, 1, 0])

In [30]:
output

array([1., 1., 1., 1., 1.])

In [18]:
level

array([0, 1, 1, 2, 3])

In [19]:
list(level).index(2)

3

pointer - вроде исправил (нужна проверка на более больших графах)

In [52]:
''' Determine the flow values from a starting node A to all other nodes in the network. 
This is done by working up from the lowest layers of the breadth-first search and dividing up the flow above a node 
in proportion to the number of shortest paths coming into it on each edge.'''
def flow(new_graph, level, root, output):
    
    n = new_graph.shape[0]
    nodes = np.array(range(n))#номера вершин для удобства
    flows = np.ones(n)
    
    for x in np.array(range(np.max(level)+1)[::-1]):
        for y in nodes[level == x]:  #не в тот массив смотрели(нужны были номера вершин!)
            for i in range(n):
                if(new_graph[i][y] >= 0 and level[i] == x-1):
                    new_graph[i][y] = flows[y] * output[i] / output[y]
                    new_graph[y][i] = flows[y] * output[i] / output[y]
                    flows[i] = flows[y] * output[i] / output[y]
        
    return new_graph

In [21]:
new_graph

array([[-1.,  0.,  0., -1., -1.],
       [ 0., -1., -1., -1., -1.],
       [ 0., -1., -1.,  0., -1.],
       [-1., -1.,  0., -1.,  0.],
       [-1., -1., -1.,  0., -1.]])

In [22]:
flow(new_graph, level, root, output)

array([[-1.,  1.,  1., -1., -1.],
       [ 1., -1., -1., -1., -1.],
       [ 1., -1., -1.,  1., -1.],
       [-1., -1.,  1., -1.,  1.],
       [-1., -1., -1.,  1., -1.]])

flow - вроде исправил (нужна проверка на более больших графах)

In [66]:
def removeEdge(graph):
    max_edge = graph.max()
    graph[graph == max_edge] = -1       
    return graph

In [109]:
def summary(graph,iterations):
    output = []
    
    for y in range(iterations):
        
        graph[graph >= 0] = 0
        for x in range(graph.shape[0]):
            root = x

            breadth_first_value = breadth_first(graph, root)
            
            node_values = pointer(breadth_first_value[0], breadth_first_value[1], root)
            
            
            flow_values = flow(
                breadth_first_value[0],
                breadth_first_value[1],
                root,
                node_values)

            graph[flow_values >= 0] += flow_values[flow_values >= 0]

        graph = removeEdge(graph)
        output.append(graph.copy())#Делаем копию
    return output

In [110]:
summary(graph,2)

[array([[-1.,  2.,  4., -1., -1.],
        [ 2., -1.,  4., -1., -1.],
        [ 4.,  4., -1., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.]]), array([[-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.],
        [-1., -1., -1., -1., -1.]])]